<a href="https://colab.research.google.com/github/myazann/Lung_Cancer/blob/main/LungCancer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git init
!git pull https://github.com/myazann/TCIA-API-SDK.git

!cp -r /content/tcia-rest-client-python/src/* /content

!rm -rf tcia-rest-client-java/
!rm -rf tcia-rest-client-python/

!cp /content/drive/MyDrive/Annotation.zip /content
!unzip Annotation.zip
!rm Annotation.zip

!pip install xmltodict

In [169]:
from tciaclient import TCIAClient
import json
import os
import numpy as np
from xml.etree import cElementTree as ElementTree
import xmltodict
from lxml import etree


In [ ]:
tc = TCIAClient(baseUrl="https://services.cancerimagingarchive.net/services/v4", resource = "TCIA")
clctn = "Lung-PET-CT-Dx"
series = json.loads(tc.get_series(collection = clctn, modality = "CT").read())

ptnt_series = {}

for elem in series:
  try:
    if elem["BodyPartExamined"] == "CHEST":
      pt_id = elem["PatientID"].split("-")[1]
      ptnt_series[elem["SeriesInstanceUID"]] = pt_id
  except Exception as e:
    print(e)

In [ ]:
folders = np.array(os.listdir("Annotation"))
folders = [i for i in folders if "A" in i]

bboxs = {}

for folder in folders:
  path = "Annotation/" + folder
  for img in os.listdir(path):
    file_path = path + "/" + img
    try:
      dt = xmltodict.parse(ElementTree.tostring(ElementTree.parse(file_path).getroot()))["annotation"]["object"]["bndbox"]
      img_name = folder + "-" + img.split(".x")[0]
      bboxs[img_name] = dt
    except Exception as e:
      print(e, file_path)